# Statistical Analysis Template for Medical Research

**Project:** [Enter project name]

**Principal Investigator:** [Enter PI name]

**Statistician:** [Enter statistician name]

**Date:** [Enter date]

**Version:** 1.0

---

## Research Questions

**Primary Research Question:**
- [State primary research question]

**Secondary Research Questions:**
- [List secondary research questions]

**Hypotheses:**
- **H₀:** [Null hypothesis]
- **H₁:** [Alternative hypothesis]

---

## Study Design and Variables

**Study Design:** [e.g., RCT, cohort, case-control, cross-sectional]

**Primary Outcome:** [Define primary outcome variable]

**Secondary Outcomes:** [List secondary outcomes]

**Predictors/Exposures:** [List key variables]

**Confounders:** [List potential confounding variables]

**Sample Size Justification:** [Include power calculation]


## 1. Setup and Data Loading

Load required packages and import the medical statistics toolkit.

In [ ]:
# Import required packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Import medical statistics toolkit
from medical_stats_toolkit import (
    DescriptiveStatistics, HypothesisTests, EffectSizes, PowerAnalysis,
    SurvivalAnalysis, MetaAnalysis, MLEvaluationMetrics, 
    MedicalVisualizations, MultipleComparisons, load_data, generate_sample_data
)

# Set visualization parameters
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

print("Medical Statistics Toolkit loaded successfully!")
print("Notebook template ready for analysis.")

### Load Data

Load your dataset and perform initial inspection.

In [ ]:
# Load your data
# Replace with your actual data file path
# data = load_data('path/to/your/data.csv')

# For demonstration, we'll generate sample medical data
data = generate_sample_data(n_samples=300, seed=42)

print(f"Dataset shape: {data.shape}")
print("\nFirst few rows:")
display(data.head())

print("\nDataset info:")
data.info()

## 2. Data Quality Assessment

Examine data quality, missing values, and outliers.

In [ ]:
# Check for missing values
missing_data = data.isnull().sum()
missing_percent = (missing_data / len(data)) * 100

missing_summary = pd.DataFrame({
    'Missing_Count': missing_data,
    'Missing_Percent': missing_percent
}).sort_values('Missing_Percent', ascending=False)

print("Missing Data Summary:")
print(missing_summary[missing_summary['Missing_Count'] > 0])

# Visualize missing data pattern
if missing_data.sum() > 0:
    plt.figure(figsize=(10, 6))
    sns.heatmap(data.isnull(), cbar=True, yticklabels=False, cmap='viridis')
    plt.title('Missing Data Pattern')
    plt.show()

In [ ]:
# Identify potential outliers for continuous variables
continuous_vars = data.select_dtypes(include=[np.number]).columns

print("Outlier Detection (values beyond 3 SD):")
for var in continuous_vars:
    if var != 'patient_id':  # Skip ID variables
        mean_val = data[var].mean()
        std_val = data[var].std()
        outliers = data[(data[var] < mean_val - 3*std_val) | (data[var] > mean_val + 3*std_val)]
        print(f"{var}: {len(outliers)} potential outliers ({len(outliers)/len(data)*100:.1f}%)")

## 3. Descriptive Statistics

Generate comprehensive descriptive statistics for all variables.

In [ ]:
# Descriptive statistics for continuous variables
print("CONTINUOUS VARIABLES")
print("=" * 50)

continuous_vars = ['age', 'continuous_outcome', 'survival_time']

descriptive_results = {}
for var in continuous_vars:
    if var in data.columns:
        stats = DescriptiveStatistics.summary_statistics(data[var])
        descriptive_results[var] = stats
        
        print(f"\n{var.upper()}:")
        print(f"  n = {stats['n']}")
        print(f"  Missing = {stats['missing']}")
        print(f"  Mean ± SD = {stats['mean']:.2f} ± {stats['std']:.2f}")
        print(f"  95% CI = [{stats['ci_lower']:.2f}, {stats['ci_upper']:.2f}]")
        print(f"  Median [IQR] = {stats['median']:.2f} [{stats['q1']:.2f}, {stats['q3']:.2f}]")
        print(f"  Range = [{stats['min']:.2f}, {stats['max']:.2f}]")
        
        # Normality test
        if not np.isnan(stats['shapiro_p']):
            normality = "Normal" if stats['normal_distribution'] else "Non-normal"
            print(f"  Distribution = {normality} (Shapiro-Wilk p = {stats['shapiro_p']:.3f})")

In [ ]:
# Descriptive statistics for categorical variables
print("\nCATEGORICAL VARIABLES")
print("=" * 50)

categorical_vars = ['gender', 'diabetes', 'treatment', 'outcome', 'event_observed']

for var in categorical_vars:
    if var in data.columns:
        cat_stats = DescriptiveStatistics.categorical_summary(data[var])
        
        print(f"\n{var.upper()}:")
        for _, row in cat_stats.iterrows():
            print(f"  {row['category']}: {row['count']} ({row['percentage']:.1f}%) "
                  f"[95% CI: {row['ci_lower']:.1f}%, {row['ci_upper']:.1f}%]")

## 4. Exploratory Data Analysis

Create visualizations to understand data distribution and relationships.

In [ ]:
# Distribution plots for continuous variables
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.ravel()

continuous_vars = ['age', 'continuous_outcome', 'survival_time']

for i, var in enumerate(continuous_vars):
    if var in data.columns and i < len(axes):
        # Histogram with density curve
        data[var].hist(bins=30, alpha=0.7, ax=axes[i], density=True)
        data[var].plot(kind='density', ax=axes[i], color='red', linewidth=2)
        axes[i].set_title(f'Distribution of {var.title()}')
        axes[i].set_ylabel('Density')

# Remove empty subplot
if len(continuous_vars) < len(axes):
    fig.delaxes(axes[-1])

plt.tight_layout()
plt.show()

In [ ]:
# Box plots by groups
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Age by treatment group
data.boxplot(column='age', by='treatment', ax=axes[0])
axes[0].set_title('Age by Treatment Group')
axes[0].set_xlabel('Treatment (0=Control, 1=Active)')

# Continuous outcome by treatment
data.boxplot(column='continuous_outcome', by='treatment', ax=axes[1])
axes[1].set_title('Continuous Outcome by Treatment Group')
axes[1].set_xlabel('Treatment (0=Control, 1=Active)')

# Continuous outcome by diabetes status
data.boxplot(column='continuous_outcome', by='diabetes', ax=axes[2])
axes[2].set_title('Continuous Outcome by Diabetes Status')
axes[2].set_xlabel('Diabetes (0=No, 1=Yes)')

plt.tight_layout()
plt.show()

In [ ]:
# Correlation matrix for continuous variables
numeric_cols = data.select_dtypes(include=[np.number]).columns
correlation_matrix = data[numeric_cols].corr()

plt.figure(figsize=(10, 8))
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(correlation_matrix, mask=mask, annot=True, cmap='coolwarm', center=0,
            square=True, fmt='.2f', cbar_kws={"shrink": .8})
plt.title('Correlation Matrix')
plt.tight_layout()
plt.show()

## 5. Primary Analysis

Conduct the main statistical analysis to address primary research questions.

In [ ]:
# Example: Compare continuous outcome between treatment groups
print("PRIMARY ANALYSIS: Treatment Effect on Continuous Outcome")
print("=" * 60)

# Separate groups
treatment_group = data[data['treatment'] == 1]['continuous_outcome'].dropna()
control_group = data[data['treatment'] == 0]['continuous_outcome'].dropna()

# Check normality assumption
treatment_normal = stats.shapiro(treatment_group)[1] > 0.05
control_normal = stats.shapiro(control_group)[1] > 0.05

print(f"Treatment group normality: {treatment_normal} (p = {stats.shapiro(treatment_group)[1]:.3f})")
print(f"Control group normality: {control_normal} (p = {stats.shapiro(control_group)[1]:.3f})")

# Parametric test (if assumptions met)
if treatment_normal and control_normal:
    print("\nUsing parametric test (Independent samples t-test):")
    t_result = HypothesisTests.t_test_independent(treatment_group, control_group)
    
    print(f"T-statistic: {t_result['t_statistic']:.4f}")
    print(f"P-value: {t_result['p_value']:.6f}")
    print(f"Degrees of freedom: {t_result['degrees_of_freedom']:.0f}")
    print(f"Cohen's d: {t_result['cohens_d']:.4f}")
    print(f"Result: {t_result['interpretation']}")
    
    # Group statistics
    print(f"\nTreatment group: Mean = {t_result['group1_stats']['mean']:.2f} ± {t_result['group1_stats']['std']:.2f}")
    print(f"Control group: Mean = {t_result['group2_stats']['mean']:.2f} ± {t_result['group2_stats']['std']:.2f}")
    print(f"Mean difference: {t_result['group1_stats']['mean'] - t_result['group2_stats']['mean']:.2f}")

# Non-parametric alternative
print("\nNon-parametric test (Mann-Whitney U test):")
mw_result = HypothesisTests.mann_whitney_u(treatment_group, control_group)
print(f"U-statistic: {mw_result['u_statistic']:.0f}")
print(f"P-value: {mw_result['p_value']:.6f}")
print(f"Effect size (r): {mw_result['effect_size_r']:.4f}")
print(f"Treatment median: {mw_result['median_group1']:.2f}")
print(f"Control median: {mw_result['median_group2']:.2f}")

In [ ]:
# Example: Analyze binary outcome (chi-square test)
print("\nBINARY OUTCOME ANALYSIS: Treatment Effect on Binary Outcome")
print("=" * 60)

# Create contingency table
contingency_table = pd.crosstab(data['treatment'], data['outcome'], margins=True)
print("Contingency Table:")
print(contingency_table)

# Chi-square test
chi2_result = HypothesisTests.chi_square_test(contingency_table.iloc[:-1, :-1])
print(f"\nChi-square statistic: {chi2_result['chi2_statistic']:.4f}")
print(f"P-value: {chi2_result['p_value']:.6f}")
print(f"Degrees of freedom: {chi2_result['degrees_of_freedom']}")
print(f"Cramer's V: {chi2_result['cramers_v']:.4f}")

# Effect sizes for 2x2 table
if contingency_table.shape == (3, 3):  # Including margins
    table_2x2 = contingency_table.iloc[:-1, :-1].values
    a, b = table_2x2[0, 1], table_2x2[0, 0]  # Treatment: outcome=1, outcome=0
    c, d = table_2x2[1, 1], table_2x2[1, 0]  # Control: outcome=1, outcome=0
    
    or_result = EffectSizes.odds_ratio_ci(a, b, c, d)
    rr_result = EffectSizes.relative_risk_ci(a, b, c, d)
    
    print(f"\nOdds Ratio: {or_result['odds_ratio']:.4f} [95% CI: {or_result['ci_lower']:.4f}, {or_result['ci_upper']:.4f}]")
    print(f"Relative Risk: {rr_result['relative_risk']:.4f} [95% CI: {rr_result['ci_lower']:.4f}, {rr_result['ci_upper']:.4f}]")

## 6. Secondary Analyses

Perform additional analyses for secondary outcomes and exploratory questions.

In [ ]:
# Subgroup analysis by diabetes status
print("SUBGROUP ANALYSIS: Treatment Effect by Diabetes Status")
print("=" * 55)

subgroup_results = []

for diabetes_status in [0, 1]:
    subgroup_data = data[data['diabetes'] == diabetes_status]
    treatment_sub = subgroup_data[subgroup_data['treatment'] == 1]['continuous_outcome'].dropna()
    control_sub = subgroup_data[subgroup_data['treatment'] == 0]['continuous_outcome'].dropna()
    
    if len(treatment_sub) > 5 and len(control_sub) > 5:  # Minimum sample size check
        t_result = HypothesisTests.t_test_independent(treatment_sub, control_sub)
        
        diabetes_label = "No Diabetes" if diabetes_status == 0 else "Diabetes"
        print(f"\n{diabetes_label}:")
        print(f"  Sample sizes: Treatment n={len(treatment_sub)}, Control n={len(control_sub)}")
        print(f"  Mean difference: {t_result['group1_stats']['mean'] - t_result['group2_stats']['mean']:.2f}")
        print(f"  T-statistic: {t_result['t_statistic']:.4f}")
        print(f"  P-value: {t_result['p_value']:.6f}")
        print(f"  Cohen's d: {t_result['cohens_d']:.4f}")
        
        subgroup_results.append({
            'subgroup': diabetes_label,
            'p_value': t_result['p_value'],
            'effect_size': t_result['cohens_d']
        })

# Store p-values for multiple comparisons correction
if subgroup_results:
    subgroup_p_values = [result['p_value'] for result in subgroup_results]
    
    # Apply Bonferroni correction
    correction_result = MultipleComparisons.adjust_p_values(subgroup_p_values, method='bonferroni')
    
    print("\nMultiple Comparisons Correction (Bonferroni):")
    for i, result in enumerate(subgroup_results):
        print(f"{result['subgroup']}: Adjusted p = {correction_result['adjusted_p_values'][i]:.6f}")

In [ ]:
# Multiple regression analysis
print("\nMULTIPLE REGRESSION ANALYSIS")
print("=" * 40)

# Prepare data for regression
regression_data = data[['continuous_outcome', 'treatment', 'age', 'gender', 'diabetes']].dropna()

# Define variables
y = regression_data['continuous_outcome']
X = regression_data[['treatment', 'age', 'gender', 'diabetes']]
X = sm.add_constant(X)  # Add intercept

# Fit model
model = sm.OLS(y, X).fit()

print("Multiple Linear Regression Results:")
print(model.summary())

# Extract key results
treatment_coef = model.params['treatment']
treatment_pvalue = model.pvalues['treatment']
treatment_ci = model.conf_int().loc['treatment']

print(f"\nTreatment effect (adjusted):")
print(f"Coefficient: {treatment_coef:.4f} [95% CI: {treatment_ci[0]:.4f}, {treatment_ci[1]:.4f}]")
print(f"P-value: {treatment_pvalue:.6f}")
print(f"R-squared: {model.rsquared:.4f}")
print(f"Adjusted R-squared: {model.rsquared_adj:.4f}")

## 7. Survival Analysis (if applicable)

Perform survival analysis if time-to-event data is available.

In [ ]:
# Kaplan-Meier survival analysis
print("SURVIVAL ANALYSIS")
print("=" * 25)

# Overall survival
km_overall = SurvivalAnalysis.kaplan_meier_analysis(
    data['survival_time'], 
    data['event_observed']
)

if 'error' not in km_overall:
    print(f"Overall median survival: {km_overall['median_survival']:.2f} time units")
    
    # Survival by treatment group
    km_by_treatment = SurvivalAnalysis.kaplan_meier_analysis(
        data['survival_time'], 
        data['event_observed'],
        groups=data['treatment']
    )
    
    if 'logrank_test' in km_by_treatment:
        logrank = km_by_treatment['logrank_test']
        print(f"\nLog-rank test p-value: {logrank['p_value']:.6f}")
        print(f"Significant difference: {logrank['significant']}")
        
        # Median survival by group
        for group in [0, 1]:
            if group in km_by_treatment:
                median_surv = km_by_treatment[group]['median_survival']
                group_label = "Control" if group == 0 else "Treatment"
                print(f"{group_label} median survival: {median_surv:.2f} time units")
    
    # Create Kaplan-Meier plot
    km_plot = MedicalVisualizations.kaplan_meier_plot(
        data['survival_time'], 
        data['event_observed'],
        groups=data['treatment'],
        title="Kaplan-Meier Survival Curves by Treatment Group"
    )
    
    if km_plot:
        plt.show()
else:
    print(km_overall['error'])

## 8. Power Analysis and Sample Size

Evaluate the statistical power of your analyses and calculate required sample sizes.

In [ ]:
# Post-hoc power analysis
print("POST-HOC POWER ANALYSIS")
print("=" * 30)

# Calculate observed effect size from primary analysis
treatment_group = data[data['treatment'] == 1]['continuous_outcome'].dropna()
control_group = data[data['treatment'] == 0]['continuous_outcome'].dropna()

observed_effect_size = EffectSizes.cohens_d(treatment_group, control_group)

# Calculate achieved power
achieved_power = PowerAnalysis.power_t_test_independent(
    effect_size=observed_effect_size,
    n1=len(treatment_group),
    n2=len(control_group),
    alpha=0.05
)

print(f"Observed effect size (Cohen's d): {observed_effect_size:.4f}")
print(f"Achieved statistical power: {achieved_power:.4f} ({achieved_power*100:.1f}%)")
print(f"Sample sizes: Treatment n={len(treatment_group)}, Control n={len(control_group)}")

# Sample size for different effect sizes and power levels
print("\nSAMPLE SIZE CALCULATIONS FOR FUTURE STUDIES")
print("=" * 50)

effect_sizes = [0.2, 0.5, 0.8]  # Small, medium, large
power_levels = [0.8, 0.9]

for power in power_levels:
    print(f"\nFor {power*100}% power:")
    for es in effect_sizes:
        sample_size = PowerAnalysis.sample_size_t_test_independent(
            effect_size=es, power=power, alpha=0.05
        )
        print(f"  Cohen's d = {es}: n = {sample_size['total_n']} total ({sample_size['n1']} per group)")

## 9. Model Diagnostics and Assumptions

Check statistical assumptions and model diagnostics.

In [ ]:
# Assumption checks for regression model
print("MODEL DIAGNOSTICS")
print("=" * 25)

# Residual analysis
if 'model' in locals():
    residuals = model.resid
    fitted_values = model.fittedvalues
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # Residuals vs fitted
    axes[0, 0].scatter(fitted_values, residuals, alpha=0.6)
    axes[0, 0].axhline(y=0, color='red', linestyle='--')
    axes[0, 0].set_xlabel('Fitted Values')
    axes[0, 0].set_ylabel('Residuals')
    axes[0, 0].set_title('Residuals vs Fitted')
    
    # Q-Q plot
    stats.probplot(residuals, dist="norm", plot=axes[0, 1])
    axes[0, 1].set_title('Q-Q Plot')
    
    # Scale-Location plot
    standardized_residuals = residuals / np.std(residuals)
    axes[1, 0].scatter(fitted_values, np.sqrt(np.abs(standardized_residuals)), alpha=0.6)
    axes[1, 0].set_xlabel('Fitted Values')
    axes[1, 0].set_ylabel('√|Standardized Residuals|')
    axes[1, 0].set_title('Scale-Location Plot')
    
    # Residual histogram
    axes[1, 1].hist(residuals, bins=20, alpha=0.7, density=True)
    # Overlay normal curve
    x = np.linspace(residuals.min(), residuals.max(), 100)
    axes[1, 1].plot(x, stats.norm.pdf(x, np.mean(residuals), np.std(residuals)), 'r-', linewidth=2)
    axes[1, 1].set_title('Residual Distribution')
    axes[1, 1].set_ylabel('Density')
    
    plt.tight_layout()
    plt.show()
    
    # Statistical tests for assumptions
    shapiro_stat, shapiro_p = stats.shapiro(residuals)
    print(f"Normality of residuals (Shapiro-Wilk): p = {shapiro_p:.6f}")
    print(f"Residuals approximately normal: {shapiro_p > 0.05}")
    
    # Homoscedasticity test (Breusch-Pagan)
    try:
        from statsmodels.stats.diagnostic import het_breuschpagan
        bp_lm, bp_lm_p, bp_f, bp_f_p = het_breuschpagan(residuals, X)
        print(f"Homoscedasticity (Breusch-Pagan): p = {bp_lm_p:.6f}")
        print(f"Constant variance assumption met: {bp_lm_p > 0.05}")
    except ImportError:
        print("Breusch-Pagan test not available")

## 10. Sensitivity Analyses

Test the robustness of findings through sensitivity analyses.

In [ ]:
# Sensitivity analysis: Different methods and assumptions
print("SENSITIVITY ANALYSES")
print("=" * 25)

sensitivity_results = []

# 1. Complete case analysis (already performed)
treatment_complete = data[data['treatment'] == 1]['continuous_outcome'].dropna()
control_complete = data[data['treatment'] == 0]['continuous_outcome'].dropna()
t_complete = HypothesisTests.t_test_independent(treatment_complete, control_complete)

sensitivity_results.append({
    'method': 'Complete case analysis',
    'n_treatment': len(treatment_complete),
    'n_control': len(control_complete),
    'p_value': t_complete['p_value'],
    'effect_size': t_complete['cohens_d']
})

# 2. Analysis excluding outliers (beyond 2 SD)
outcome_mean = data['continuous_outcome'].mean()
outcome_std = data['continuous_outcome'].std()
outlier_threshold = 2

data_no_outliers = data[
    (data['continuous_outcome'] >= outcome_mean - outlier_threshold * outcome_std) &
    (data['continuous_outcome'] <= outcome_mean + outlier_threshold * outcome_std)
]

treatment_no_outliers = data_no_outliers[data_no_outliers['treatment'] == 1]['continuous_outcome'].dropna()
control_no_outliers = data_no_outliers[data_no_outliers['treatment'] == 0]['continuous_outcome'].dropna()

if len(treatment_no_outliers) > 10 and len(control_no_outliers) > 10:
    t_no_outliers = HypothesisTests.t_test_independent(treatment_no_outliers, control_no_outliers)
    sensitivity_results.append({
        'method': 'Excluding outliers (>2 SD)',
        'n_treatment': len(treatment_no_outliers),
        'n_control': len(control_no_outliers),
        'p_value': t_no_outliers['p_value'],
        'effect_size': t_no_outliers['cohens_d']
    })

# 3. Non-parametric test (Mann-Whitney U)
mw_result = HypothesisTests.mann_whitney_u(treatment_complete, control_complete)
sensitivity_results.append({
    'method': 'Mann-Whitney U test',
    'n_treatment': len(treatment_complete),
    'n_control': len(control_complete),
    'p_value': mw_result['p_value'],
    'effect_size': mw_result['effect_size_r']
})

# Display sensitivity analysis results
print("\nSensitivity Analysis Summary:")
for result in sensitivity_results:
    print(f"\n{result['method']}:")
    print(f"  Sample sizes: Treatment n={result['n_treatment']}, Control n={result['n_control']}")
    print(f"  P-value: {result['p_value']:.6f}")
    print(f"  Effect size: {result['effect_size']:.4f}")
    print(f"  Significant: {result['p_value'] < 0.05}")

# Check consistency of results
significant_count = sum(1 for result in sensitivity_results if result['p_value'] < 0.05)
print(f"\nConsistency check: {significant_count}/{len(sensitivity_results)} analyses significant")

if significant_count == len(sensitivity_results):
    print("Results are robust across all sensitivity analyses.")
elif significant_count == 0:
    print("No significant effect found in any sensitivity analysis.")
else:
    print("Mixed results - findings may be sensitive to analytical choices.")

## 11. Results Summary

Comprehensive summary of all analyses and key findings.

In [ ]:
# Create comprehensive results table
print("COMPREHENSIVE RESULTS SUMMARY")
print("=" * 40)

# Primary outcome results
if 't_complete' in locals():
    primary_result = t_complete
    
    print("\nPRIMARY ANALYSIS RESULTS:")
    print("-" * 30)
    print(f"Outcome: Continuous Outcome")
    print(f"Analysis: Independent samples t-test")
    print(f"Sample size: n = {primary_result['group1_stats']['n'] + primary_result['group2_stats']['n']}")
    print(f"Treatment group: n = {primary_result['group1_stats']['n']}, Mean = {primary_result['group1_stats']['mean']:.2f} ± {primary_result['group1_stats']['std']:.2f}")
    print(f"Control group: n = {primary_result['group2_stats']['n']}, Mean = {primary_result['group2_stats']['mean']:.2f} ± {primary_result['group2_stats']['std']:.2f}")
    
    mean_diff = primary_result['group1_stats']['mean'] - primary_result['group2_stats']['mean']
    print(f"Mean difference: {mean_diff:.2f}")
    print(f"Cohen's d: {primary_result['cohens_d']:.4f}")
    print(f"95% CI for difference: [{primary_result['group1_stats']['ci_lower'] - primary_result['group2_stats']['ci_upper']:.2f}, {primary_result['group1_stats']['ci_upper'] - primary_result['group2_stats']['ci_lower']:.2f}]")
    print(f"T-statistic: {primary_result['t_statistic']:.4f}")
    print(f"P-value: {primary_result['p_value']:.6f}")
    print(f"Statistical significance: {primary_result['significant']}")
    
    # Effect size interpretation
    abs_d = abs(primary_result['cohens_d'])
    if abs_d < 0.2:
        effect_interpretation = "negligible"
    elif abs_d < 0.5:
        effect_interpretation = "small"
    elif abs_d < 0.8:
        effect_interpretation = "medium"
    else:
        effect_interpretation = "large"
    
    print(f"Effect size magnitude: {effect_interpretation}")
    
    # Clinical significance (if thresholds defined)
    clinical_threshold = 5.0  # Example threshold - adjust based on your outcome
    clinically_significant = abs(mean_diff) >= clinical_threshold
    print(f"Clinically meaningful difference (≥{clinical_threshold}): {clinically_significant}")

# Power analysis summary
if 'achieved_power' in locals():
    print(f"\nPOWER ANALYSIS:")
    print("-" * 20)
    print(f"Achieved power: {achieved_power:.3f} ({achieved_power*100:.1f}%)")
    power_adequate = achieved_power >= 0.8
    print(f"Adequate power (≥80%): {power_adequate}")

# Limitations and considerations
print(f"\nSTATISTICAL CONSIDERATIONS:")
print("-" * 30)

considerations = []
if 'primary_result' in locals():
    if not primary_result['group1_stats']['normal_distribution'] or not primary_result['group2_stats']['normal_distribution']:
        considerations.append("Non-normal distribution detected - consider non-parametric alternatives")
    
    if primary_result['group1_stats']['n'] < 30 or primary_result['group2_stats']['n'] < 30:
        considerations.append("Small sample size - results should be interpreted with caution")

if 'achieved_power' in locals() and achieved_power < 0.8:
    considerations.append("Statistical power < 80% - risk of Type II error")

if considerations:
    for i, consideration in enumerate(considerations, 1):
        print(f"{i}. {consideration}")
else:
    print("No major statistical concerns identified")

## 12. Publication-Ready Tables and Figures

Generate publication-quality tables and figures for manuscript submission.

In [ ]:
# Table 1: Baseline characteristics
print("TABLE 1: BASELINE CHARACTERISTICS")
print("=" * 40)

def create_baseline_table(data):
    """Create a baseline characteristics table."""
    
    table_data = []
    
    # Overall column
    overall_col = []
    control_col = []
    treatment_col = []
    p_values = []
    
    # Sample sizes
    n_total = len(data)
    n_control = len(data[data['treatment'] == 0])
    n_treatment = len(data[data['treatment'] == 1])
    
    # Age
    age_overall = DescriptiveStatistics.summary_statistics(data['age'])
    age_control = DescriptiveStatistics.summary_statistics(data[data['treatment'] == 0]['age'])
    age_treatment = DescriptiveStatistics.summary_statistics(data[data['treatment'] == 1]['age'])
    age_ttest = HypothesisTests.t_test_independent(
        data[data['treatment'] == 1]['age'],
        data[data['treatment'] == 0]['age']
    )
    
    overall_col.append(f"{age_overall['mean']:.1f} ± {age_overall['std']:.1f}")
    control_col.append(f"{age_control['mean']:.1f} ± {age_control['std']:.1f}")
    treatment_col.append(f"{age_treatment['mean']:.1f} ± {age_treatment['std']:.1f}")
    p_values.append(age_ttest['p_value'])
    
    # Gender
    gender_overall = DescriptiveStatistics.categorical_summary(data['gender'])
    gender_control = DescriptiveStatistics.categorical_summary(data[data['treatment'] == 0]['gender'])
    gender_treatment = DescriptiveStatistics.categorical_summary(data[data['treatment'] == 1]['gender'])
    
    # Chi-square test for gender
    gender_crosstab = pd.crosstab(data['treatment'], data['gender'])
    gender_chi2 = HypothesisTests.chi_square_test(gender_crosstab)
    
    # Assuming 1 = female
    female_overall = gender_overall[gender_overall['category'] == 1]['count'].iloc[0] if len(gender_overall[gender_overall['category'] == 1]) > 0 else 0
    female_control = gender_control[gender_control['category'] == 1]['count'].iloc[0] if len(gender_control[gender_control['category'] == 1]) > 0 else 0
    female_treatment = gender_treatment[gender_treatment['category'] == 1]['count'].iloc[0] if len(gender_treatment[gender_treatment['category'] == 1]) > 0 else 0
    
    overall_col.append(f"{female_overall} ({female_overall/n_total*100:.1f}%)")
    control_col.append(f"{female_control} ({female_control/n_control*100:.1f}%)")
    treatment_col.append(f"{female_treatment} ({female_treatment/n_treatment*100:.1f}%)")
    p_values.append(gender_chi2['p_value'])
    
    # Diabetes
    diabetes_crosstab = pd.crosstab(data['treatment'], data['diabetes'])
    diabetes_chi2 = HypothesisTests.chi_square_test(diabetes_crosstab)
    
    diabetes_overall = len(data[data['diabetes'] == 1])
    diabetes_control = len(data[(data['treatment'] == 0) & (data['diabetes'] == 1)])
    diabetes_treatment = len(data[(data['treatment'] == 1) & (data['diabetes'] == 1)])
    
    overall_col.append(f"{diabetes_overall} ({diabetes_overall/n_total*100:.1f}%)")
    control_col.append(f"{diabetes_control} ({diabetes_control/n_control*100:.1f}%)")
    treatment_col.append(f"{diabetes_treatment} ({diabetes_treatment/n_treatment*100:.1f}%)")
    p_values.append(diabetes_chi2['p_value'])
    
    # Create table
    baseline_table = pd.DataFrame({
        'Characteristic': ['Age, years (mean ± SD)', 'Female gender, n (%)', 'Diabetes, n (%)'],
        f'Overall (n={n_total})': overall_col,
        f'Control (n={n_control})': control_col,
        f'Treatment (n={n_treatment})': treatment_col,
        'P-value': [f"{p:.3f}" if p >= 0.001 else "<0.001" for p in p_values]
    })
    
    return baseline_table

baseline_table = create_baseline_table(data)
print(baseline_table.to_string(index=False))

# Save to CSV for manuscript use
baseline_table.to_csv('baseline_characteristics_table.csv', index=False)
print("\nBaseline table saved as 'baseline_characteristics_table.csv'")

In [ ]:
# Create publication-quality figures
print("\nCREATING PUBLICATION FIGURES")
print("=" * 35)

# Set publication style
plt.style.use('default')
plt.rcParams.update({
    'font.size': 12,
    'axes.linewidth': 1.2,
    'xtick.major.width': 1.2,
    'ytick.major.width': 1.2,
    'figure.dpi': 300
})

# Figure 1: Primary outcome comparison
fig, ax = plt.subplots(figsize=(8, 6))

# Box plot with individual points
box_data = [data[data['treatment'] == 0]['continuous_outcome'].dropna(),
            data[data['treatment'] == 1]['continuous_outcome'].dropna()]

bp = ax.boxplot(box_data, labels=['Control', 'Treatment'], patch_artist=True, 
                boxprops=dict(facecolor='lightblue', alpha=0.7),
                medianprops=dict(color='red', linewidth=2))

# Add individual points with jitter
for i, group_data in enumerate(box_data):
    x_jitter = np.random.normal(i+1, 0.05, size=len(group_data))
    ax.scatter(x_jitter, group_data, alpha=0.4, s=20, color='darkblue')

ax.set_ylabel('Continuous Outcome')
ax.set_xlabel('Treatment Group')
ax.set_title('Primary Outcome by Treatment Group')
ax.grid(True, alpha=0.3)

# Add statistical annotation
if 'primary_result' in locals():
    y_max = max([max(group) for group in box_data])
    ax.text(1.5, y_max * 1.1, f"p = {primary_result['p_value']:.3f}" if primary_result['p_value'] >= 0.001 else "p < 0.001",
           ha='center', fontweight='bold')
    ax.text(1.5, y_max * 1.05, f"Cohen's d = {primary_result['cohens_d']:.3f}",
           ha='center')

plt.tight_layout()
plt.savefig('figure1_primary_outcome.png', dpi=300, bbox_inches='tight')
plt.show()

print("Figure 1 saved as 'figure1_primary_outcome.png'")

## 13. Statistical Conclusions and Recommendations

Final statistical interpretation and recommendations for the research team.

In [ ]:
# Final conclusions and recommendations
print("STATISTICAL CONCLUSIONS AND RECOMMENDATIONS")
print("=" * 50)

print("\n1. PRIMARY FINDINGS:")
print("-" * 20)
if 'primary_result' in locals():
    if primary_result['significant']:
        print(f"✓ Statistically significant treatment effect detected (p = {primary_result['p_value']:.3f})")
        print(f"✓ Effect size: {effect_interpretation} (Cohen's d = {primary_result['cohens_d']:.3f})")
        
        if clinically_significant:
            print(f"✓ Clinically meaningful difference observed (|difference| = {abs(mean_diff):.2f})")
        else:
            print(f"⚠ Statistical significance achieved but clinical meaningfulness questionable")
    else:
        print(f"✗ No statistically significant treatment effect (p = {primary_result['p_value']:.3f})")
        print(f"  Effect size: {effect_interpretation} (Cohen's d = {primary_result['cohens_d']:.3f})")

print("\n2. STUDY VALIDITY:")
print("-" * 20)
validity_issues = []

# Check sample size adequacy
if 'achieved_power' in locals():
    if achieved_power >= 0.8:
        print("✓ Adequate statistical power achieved (≥80%)")
    else:
        print(f"⚠ Insufficient statistical power ({achieved_power*100:.1f}%) - risk of Type II error")
        validity_issues.append("Underpowered study")

# Check for multiple comparisons
if 'subgroup_results' in locals() and len(subgroup_results) > 1:
    print("✓ Multiple comparisons correction applied")
else:
    print("ℹ Single primary comparison - no multiple testing correction needed")

# Check assumptions
if 'primary_result' in locals():
    if primary_result['group1_stats']['normal_distribution'] and primary_result['group2_stats']['normal_distribution']:
        print("✓ Normality assumptions met for parametric testing")
    else:
        print("⚠ Normality assumptions violated - non-parametric alternatives considered")

print("\n3. RECOMMENDATIONS FOR MANUSCRIPT:")
print("-" * 35)

recommendations = [
    "Report both statistical and clinical significance",
    "Include confidence intervals for all effect estimates",
    "Present both parametric and non-parametric test results if assumptions violated",
    "Acknowledge study limitations in discussion"
]

if validity_issues:
    recommendations.extend([
        "Discuss power limitations and risk of Type II error",
        "Suggest appropriate sample size for future studies"
    ])

if 'sensitivity_results' in locals():
    recommendations.append("Include sensitivity analysis results to demonstrate robustness")

for i, rec in enumerate(recommendations, 1):
    print(f"{i}. {rec}")

print("\n4. SUGGESTED FUTURE RESEARCH:")
print("-" * 30)

future_research = []

if 'achieved_power' in locals() and achieved_power < 0.8:
    future_research.append(f"Adequately powered replication study (n ≥ {sample_size['total_n']} for 80% power)")

if 'subgroup_results' in locals():
    future_research.append("Dedicated subgroup analysis with stratified randomization")

if not clinically_significant:
    future_research.append("Define minimal clinically important difference before future trials")

future_research.extend([
    "Longer-term follow-up to assess sustainability of effects",
    "Cost-effectiveness analysis of intervention",
    "Mechanistic studies to understand biological pathways"
])

for i, research in enumerate(future_research, 1):
    print(f"{i}. {research}")

print("\n" + "=" * 50)
print("STATISTICAL ANALYSIS COMPLETE")
print("Analysis conducted using Medical Statistics Toolkit v1.0")
print(f"Analysis date: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M')}")
print("=" * 50)

## Appendix: Analysis Log and Session Info

Document the analysis environment and session details for reproducibility.

In [ ]:
# Session information for reproducibility
import sys
import platform
from datetime import datetime

print("ANALYSIS SESSION INFORMATION")
print("=" * 35)
print(f"Analysis date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Python version: {sys.version}")
print(f"Platform: {platform.system()} {platform.release()}")

# Package versions
import importlib
packages = ['numpy', 'pandas', 'scipy', 'matplotlib', 'seaborn', 'statsmodels', 'sklearn']

print("\nPackage versions:")
for package in packages:
    try:
        module = importlib.import_module(package)
        version = getattr(module, '__version__', 'Unknown')
        print(f"  {package}: {version}")
    except ImportError:
        print(f"  {package}: Not installed")

# Dataset summary
if 'data' in locals():
    print(f"\nDataset information:")
    print(f"  Shape: {data.shape}")
    print(f"  Missing values: {data.isnull().sum().sum()}")
    print(f"  Memory usage: {data.memory_usage(deep=True).sum() / 1024:.1f} KB")

print("\nAnalysis completed successfully.")
print("All results saved to working directory.")